# Exercise 2, Kalman Filter Applied to a Pendulum

## Lab Instructions
Your answers for **tasks a - c** and **task g** should be written **in this notebook**.
Your answers for **tasks d - f** should be written **in your solution
 pdf file**.

You shouldn't need to write or modify any other files.

**You should execute every block of code to not miss any dependency.**

This exercise was developed by Philipp Dahlinger for the KIT Cognitive Systems Lecture, Juli 2022. The pygame implementation of the pendulum environment was adapted from this source: [Pastebin](https://pastebin.com/zTZVi8Yv)


In this notebook you should implement a Kalman Filter as well as an Extended Kalman Filter. To test your implementation we will use a pendulum with noisy observations. Read the instructions carefully and complete the unfinished functions.

Detailed instructions:

0. You may need to install pygame to run this notebook. Either you install it globally by opening a terminal and type in "pip3 install -U pygame" or you create a virtual environment (venv) and install pygame there. If you use a virtual environment, you may need to create a new kernel for the notebook. Detailled instructions for that can be found here: [StackOverflow](https://stackoverflow.com/questions/33496350/execute-python-script-within-jupyter-notebook-using-a-specific-virtualenv).

1. In script "pendulum.py", you can find the definition of the "PendulumGame" class. The only relevant quantity of the pendulum that we model is the angle. 
2. Keywords for the pendulum:
    - **angle**: The angle of the actuation of the pendulum measured between the string of the pendulum and the position of rest in the horizontal center.  <br>
    - **vel**: Angular velocity  <br>
    - **acc**: Angular acceleration <br>
    - **gravity**: This controls the acceleration of the simulation.     
    <br>
3. Our state for the Kalman filter is the current angle and the angular velocity, hence a 2-dimensional vector.



In [1]:
# DO NOT MODIFY THIS BLOCK

# Imports
from ex2.pendulum import PendulumGame
import numpy as np

pygame 2.1.2 (SDL 2.0.16, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# a) Implementation of the Kalman Filter <br>
Please finish the function **"kalman_step"** below. This function performs one update of the Kalman filter (prediction step and observation step). You can test your implementation with the cell below. For completeness sake, here is the algorithm in mathematical notation:<br>

The current belief is given by 
$$
b_t(z) = \mathcal{N}(\mu_t, \Sigma_t).
$$
The first step is the preciction step:
$$
\mu_{t+1}^- = A_t \mu_t + a_t
$$
$$
\Sigma_{t+1}^- = A_t \Sigma_t A_t^T + R_t
$$
The second step is the observation step:
$$
K_t = \Sigma_{t+1}^- H_t^T (H_t \Sigma_{t+1}^- H_t^T + Q_t)^{-1}
$$
$$
\mu_{t+1} = \mu_{t+1}^- + K_t(y_t - H_t \mu_{t+1}^-)
$$
$$
\Sigma_{t+1} = \Sigma_{t+1}^- - K_t H_t \Sigma_{t+1}^-
$$


In [2]:
# TODO: implement the function body

def kalman_step(obs, belief_mean, belief_var, A, a, H, R, Q):
    """
    obs: real valued observation of the angle of the pendulum.
    belief_mean: np.array with shape (2,) which displays the current belief about the angular position and velocity (mu_t).
    belief_var: np.array with shape (2, 2). Variance matrix of the belief gaussian (Sigma_t).
    A: np.array with shape (2, 2). Current Transition matrix A_t
    a: np.array with shape (2,). Current Offset of the transition model a_t.
    H: np.array with shape (2,). Current Measurement matrix H_t.
    R: np.array with shape (2, 2). Current Transition noise matrix R_t.
    Q: real valued observation noise Q_t.
    
    return: new_belief_mean, new_belief_var
    """
    # 1. prediction step
    mean_minus = A @ belief_mean + a
    var_minus = A @ belief_var @ A.T + R
    # 2. observation step
    K_t = (var_minus @ H.T * 1 / (H @ var_minus @ H.T + Q)).squeeze()
    new_belief_mean = mean_minus + K_t * (obs - H @ mean_minus)
    new_belief_var = var_minus - np.outer(K_t, H.squeeze()) @ var_minus

    assert new_belief_mean.shape == belief_mean.shape
    assert new_belief_var.shape == belief_var.shape
    return new_belief_mean, new_belief_var

In [3]:
# DO NOT MODIFY THIS BLOCK
# this is a test to verify that your implementation is correct. The result with the code from the solution is 
# given as a comment below. 

# some dummy values
test_obs = 1.0
test_belief_mean = np.array([1.0, 2.0])
test_belief_var = np.eye(2) * 0.5
test_A = np.ones((2, 2)) * 0.3
test_a = np.array([0.5, 2.0])
test_H = np.array([1.0, 0.0])
test_R = np.eye(2) * 0.2
test_Q = 10.

new_belief_mean, new_belief_var = kalman_step(test_obs, test_belief_mean, test_belief_var, test_A, test_a, test_H, test_R, test_Q)

print(f"{new_belief_mean=}")
print(f"{new_belief_var=}")

# Correct results:

# new_belief_mean=array([1.38872692, 2.89650146])
# new_belief_var=array([[0.28182702, 0.08746356],
#        [0.08746356, 0.28921283]])


new_belief_mean=array([1.38872692, 2.89650146])
new_belief_var=array([[0.28182702, 0.08746356],
       [0.08746356, 0.28921283]])


### b) First Order Dynamics model <br>
We model our state of the pendulum by the current angle position $x_t$ and its angular velocity $\dot x_t$
Our simple dynamics model is given by

$$
\begin{bmatrix} x_{t+1}   \\  \dot{x}_{t+1} \end{bmatrix} = \begin{bmatrix} 1 & dt \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x_{t}   \\  \dot{x}_{t} \end{bmatrix} + \begin{bmatrix} 0   \\  0 \end{bmatrix}
$$

Please finish the function **"get_first_order_system"** below. This function returns the transition matrix $A \in \mathbb{R}^{2 \times 2}$ and the linear offset $a \in \mathbb{R}^2$. 

In [4]:
# TODO: implement the function body

def get_first_order_system(dt):
    """
    dt: real value. Difference in time between 2 measurements.
    returns: Transition matrix A (np.array with shape (2, 2)) and linear offset a (np.array with shape (2,)).
    """
    A = np.array([[1, dt], [0, 1]])
    a = np.array([0, 0., ])
    return A, a

### c)Visual Analysis of the Kalman Filter<br>
Execute the cell below. You will see an animation with 3 pendulums of differenct color. 
- The red pendulum is the true state pendulum which we would like to approximate.
- The green pendulum is the visualization of the Kalman filter mean belief angle.
- the blue pendulum shows the individual noisy observations in each step. <br>

**The different lengths of the pendulums are not relevant, they are just chosen for a better visibility. The only relevant quantities are the angles of the pendulums.**  <br>

The estimated state of the Kalman filter is (even with correct implementaiton) not optimal. Describe the behavior of the green estimated pendulum. Also, explain the reason behind that specific movement. You can answer these questions inside the notebook below the next cell. <br>

In [5]:
# TODO: Run the cell

# set up the visualization
game = PendulumGame(gravity=25., framerate=60, damping=1.0, obs_noise_std=0.3)
# initial belief state
kf_belief_mean = np.array([game.angle, 0.0])
kf_belief_var = np.eye(2)

A, a = get_first_order_system(game.dt)
H = np.array([[1., 0]])
# noise values
R = np.eye(2) * 5.
Q = np.array([150.])

game.start_gui()
for step in range(10000):
    obs = game.step_physics()
    
    kf_belief_mean, kf_belief_var = kalman_step(obs, kf_belief_mean, kf_belief_var, A, a, H, R, Q)
    game.set_kf_angle(kf_belief_mean[0])
    terminated = game.redraw()
    if terminated:
        break

### Answer to c)
**Beobachtung**: Der Kalman Filter ist zeitlich versetzt und hinkt dem echten Pendel hinterher. Außerdem gibt es noch immer etwas Rauschen im gefilterten Signal.

**Erklärung**: Der Kalman Filter geht in seinem Dymanikmodell davon aus, dass auf dem Pendel keine Kräfte und daraus resultierenden Beschleunigungen wirken. Dies ist in der Realität aber nicht der Fall: Das Pendel beschleunigt und bremst. Wenn das Pendel von der linken Seite nach rechts schwingt, beschleunigt es auf dem Weg nach unten. Der Kalman Filter reagiert da zu spät, da er mit dieser Dynamik nicht rechnet. Dadurch verzögert sich das gefilterte Signal. Schwingt nun das Pendel nach rechts und bremst auf dem Weg nach oben, reagiert der Kalman Filter ebenfalls verzögert, da er weiterhin von einer gleichförmigen Bewegung ausgeht. Dadurch überschwingt das Pendel vom Kalman Filter. Insgesamt entsteht so eine zeitliche Verzögerung. 



### d) Taylor Approximation of the real Pendulum Dynamics<br>
Please answer the question e) given **on the exercise sheet** in your solution pdf. <br>

### e) Linearized Dynamics model<br>
Please answer the question f) given **on the exercise sheet** in your solution pdf. <br>

### f) Implementation of the Linearized Dynamics model<br>
Now that you have derived the dynamics model in e), implement it in the cell below. Afterwards, run the visualization of the extended Kalman filter! <br>

In [6]:
# TODO: implement the function body

def get_linearization(dt, gravity, ekf_belief_mean):
    """
    dt: real value. Difference in time between 2 measurements.
    gravity: real value. This controls the acceleration of the pendulum. (In the formula the variable 'g')
    ekf_belief_mean: np.array with shape (2,) which displays the current belief about the angular position
                     and velocity of the extended Kalman Filter.
    returns: Transition matrix A_t (np.array with shape (2, 2)) and linear offset a_t (np.array with shape (2,))
             for the timestep t.
    """
    A_t = np.array([[1, dt], [- gravity * np.cos(ekf_belief_mean[0]) * dt, 1]])
    a_t = np.array([ekf_belief_mean[0] + ekf_belief_mean[1] * dt, ekf_belief_mean[1] - 
                    gravity * np.sin(ekf_belief_mean[0]) * dt]) - A_t @ ekf_belief_mean
    return A_t, a_t

In [7]:
# TODO: Run the cell

game = PendulumGame(gravity=25., framerate=60, damping=1.0, obs_noise_std=0.3)
ekf_belief_mean = np.array([game.angle, 0.0])
ekf_belief_var = np.eye(2)

H = np.array([[1., 0]])
R = np.eye(2) * 1.
Q = np.array([80.])

game.start_gui()
for step in range(10000):
    obs = game.step_physics()
    A_t, a_t = get_linearization(game.dt, game.gravity, ekf_belief_mean)
    ekf_belief_mean, ekf_belief_var = kalman_step(obs, ekf_belief_mean, ekf_belief_var, A_t, a_t, H, R, Q)
    game.set_kf_angle(ekf_belief_mean[0])
    terminated = game.redraw()
    if terminated:
        break

### g) Voluntary exercise: Selecting good noise values<br>
The Kalman Filter is sensitive to the choice of the noise values $R$ and $Q$. Play around with these values for the Kalman filter and the extended Kalman filter. To not mess up your working implementation above, we copied the previous cell below. What is the best fit you can get?<br>

In [8]:
# TODO: Edit R and Q and see how they influence the Kalman filter

game = PendulumGame(gravity=25., framerate=60, damping=1.0, obs_noise_std=0.3)
ekf_belief_mean = np.array([game.angle, 0.0])
ekf_belief_var = np.eye(2)

H = np.array([[1., 0]])

# TODO: change these values and see what they do
R = np.eye(2) * 1.
Q = np.array([80.])

game.start_gui()
for step in range(10000):
    obs = game.step_physics()
    A_t, a_t = get_linearization(game.dt, game.gravity, ekf_belief_mean)
    ekf_belief_mean, ekf_belief_var = kalman_step(obs, ekf_belief_mean, ekf_belief_var, A_t, a_t, H, R, Q)
    game.set_kf_angle(ekf_belief_mean[0])
    terminated = game.redraw()
    if terminated:
        break